<a href="https://colab.research.google.com/github/GiovanniSorice/Deep_Music_Generator/blob/main/notebooks/Music_Generation_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer Music Generator 



In this notebook, we use an Transformer to generate some music.


**This notebook was inspired (and part of the code comes from it) by [Music_Generation_LSTM](https://colab.research.google.com/drive/19TQqekOlnOSW36VCL8CPVEQKBBukmaEQ#scrollTo=DDOBVWULXfpz)**


In [3]:
pip install compressive_transformer_pytorch

In [58]:
import torch
from compressive_transformer_pytorch import CompressiveTransformer
from compressive_transformer_pytorch import AutoregressiveWrapper
from torchsummary import summary
import tqdm
from torch.utils.data import DataLoader, Dataset



**Load dependencies**

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import utils
import os
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt
import glob
import pickle
from music21 import converter, instrument, stream, note, chord

In [6]:
# Set to false if you are not running
# this notebook in Google Colaboratory
run_on_colab = True

**Set hyperparameters**

In [19]:
# output directory name:
output_dir = 'model_output/LSTM'

# training:
epochs = 100
batch_size = 64

# vector-space embedding: 
n_dim = 64 
sequence_length = 16

# LSTM layer architecture:
n1_lstm = 256 
drop1_lstm = 0.2
n2_lstm = 256 
drop2_lstm = 0.2
n3_lstm = 256 
drop3_lstm = 0.2

# dense layer architecture: 
n_dense = 128
drop_dense = 0.2

**Google drive configuration (only Colab)**

In [8]:
if(run_on_colab):
  from google.colab import drive
  # This will prompt for authorization.
  drive.mount('/content/drive')

Mounted at /content/drive


**Load data** \\
Original MIDI files
 I have obtained  **MIDI files** from [The Lakh MIDI Dataset v0.1](https://colinraffel.com/projects/lmd/). 

## Processing data

Let's process the files, and load them into **music21**

In [9]:
file = "/content/drive/My Drive/ISPR_project/midi_songs/Andra tutto bene ('58).1.mid"
midi = converter.parse(file)
notes_to_parse = midi.flat.notes
for element in notes_to_parse[:10]:
  print(element, element.offset)

<music21.chord.Chord F3 F2> 4.0
<music21.note.Note A> 4.0
<music21.chord.Chord B1 F#3 F#2> 4.0
<music21.note.Note F> 4.0
<music21.chord.Chord C4 F4> 4.0
<music21.chord.Chord F#3 C#6 F#2> 4.5
<music21.note.Note C#> 4.75
<music21.chord.Chord F#2 E2 F#3> 5.0
<music21.chord.Chord A4 A3 F4 C4 A3> 5.0
<music21.note.Note F> 5.0


I will process all MIDI files obtaining data from each note of chord.

- If I process a **note**, I will store in the list a string representing the pitch (the note name) and the octave.

- If I process a **chord** (Remember that chords are set of notes that are played at the same time) I will store a different type of string with numbers separated by dots. Each number represents the pitch of a chord note. 

As you can see, **I are not considering yet time offsets of each element**. In this first version, we won't consider them, so all the notes and chords will have the same duration. Maybe, in the future, I will consider them.

I are creating a big list with all the elements of all the compositions.

In [10]:
notes = []
for i,file in enumerate(glob.glob("/content/drive/My Drive/ISPR_project/midi_songs/*.mid")):
  midi = converter.parse(file)
  print('\r', 'Parsing file ', i, " ",file, end='')
  notes_to_parse = None
  try: # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes
  for element in notes_to_parse:
    if isinstance(element, note.Note):
      notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
      notes.append('.'.join(str(n) for n in element.normalOrder))
with open('notes', 'wb') as filepath:
  pickle.dump(notes, filepath)

 Parsing file  3   /content/drive/My Drive/ISPR_project/midi_songs/Andra tutto bene ('58).1.mid

I obtain the number of different notes in our dataset, because this will be the **number of possible output classes**  of our model.

In [11]:
# Count different possible outputs
n_vocab = (len(set(notes)))
n_vocab

71

**Preprocess data** \\
Now, there is some **data processing** that I have to do:

- I will map each pitch or chord to an integer
- I will create pairs of input sequences and its corresponding output note

I can try different **sequence_length** to obtain different results. In this first version, I will use a sequence_length of 100.

The network will made its prediction of the next note (or chord), based on the previous *sequence_length* notes (or chords). 


In [63]:
class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len, segments):
        super().__init__()
        self.data = data
        self.seq_len = seq_len
        self.segments = segments
        self.total_len = seq_len * segments

    def __getitem__(self, index):
        rand_start = torch.randint(0, self.data.size(0) - self.total_len - 1, (1,))
        full_seq = self.data[rand_start: rand_start + self.total_len + 1].long()
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0) // self.total_len

In [146]:
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
  # Map pitches of sequence_in to integers
  network_input.append([note_to_int[char] for char in notes[i:i + sequence_length]])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns,1, sequence_length))
# normalize input
#network_input = network_input / float(n_vocab)

Let's see the new metwork_input size

In [138]:
network_input.shape

(4987, 1, 16)

**Design neural network architecture** 

In [235]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = CompressiveTransformer(
    num_tokens = n_vocab,
    dim = sequence_length,
    depth = 6,
    seq_len = 1024,
    mem_len = 1024,
    cmem_len = 256,
    cmem_ratio = 4,
    memory_layers = [5,6]
    )

    model = AutoregressiveWrapper(model)
    model.cuda()
    return model

In [236]:
model = create_network(network_input,n_vocab)

print(model)
#for loss, aux_loss, _ in model(inputs, return_loss = True):
#    (loss + aux_loss).backward()


AutoregressiveWrapper(
  (net): CompressiveTransformer(
    (token_emb): Embedding(71, 16)
    (to_model_dim): Identity()
    (to_logits): Sequential(
      (0): Identity()
      (1): Linear(in_features=16, out_features=71, bias=True)
    )
    (attn_layers): ModuleList(
      (0): GRUGating(
        (fn): PreNorm(
          (norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (fn): SelfAttention(
            (compress_mem_fn): ConvCompress(
              (conv): Conv1d(16, 16, kernel_size=(4,), stride=(4,))
            )
            (to_q): Linear(in_features=16, out_features=16, bias=False)
            (to_kv): Linear(in_features=16, out_features=32, bias=False)
            (to_out): Linear(in_features=16, out_features=16, bias=True)
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (dropout): Dropout(p=0.0, inplace=False)
            (reconstruction_attn_dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (gru): GRUCell(16, 16)

In [219]:
def cycle(loader):
    while True:
        for data in loader:
            yield data


data_train = torch.from_numpy(network_input).cuda()
train_dataset = TextSamplerDataset(data_train, sequence_length, n_patterns)
train_loader  = cycle(DataLoader(train_dataset, batch_size = 32))

In [140]:
# optimizer

optim = torch.optim.Adam(model.parameters(), lr=1e-4)

In [245]:
print(data_train[0])


tensor([[49, 56, 67, 28,  4, 12, 59, 12, 59, 12,  4, 12, 11, 12, 59, 12]],
       device='cuda:0')


In [255]:
n_patterns

4987

In [258]:
# training
for i in tqdm.tqdm(range(n_batches), mininterval=10., desc='training'):
    model.train()

    grad_accum_every = 1

    for k in range(31):
      for mlm_loss, aux_loss, is_last in model(data_train[k], max_batch_size = 16, return_loss = True):
          loss = mlm_loss + aux_loss
          (loss / grad_accum_every).backward()

          print(f'training loss: {mlm_loss.item():.4f} | aux_loss: {aux_loss.item():.4f}')

          if k == 30:
              print(k)
              torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
              optim.step()
              optim.zero_grad()








training:   0%|          | 0/157 [00:00<?, ?it/s]

training loss: 4.2881 | aux_loss: 0.0000
training loss: 4.2807 | aux_loss: 0.0000
training loss: 4.2906 | aux_loss: 0.0000
training loss: 4.2895 | aux_loss: 0.0000
training loss: 4.2849 | aux_loss: 0.0000
training loss: 4.2875 | aux_loss: 0.0000
training loss: 4.2856 | aux_loss: 0.0000
training loss: 4.2875 | aux_loss: 0.0000
training loss: 4.2856 | aux_loss: 0.0000
training loss: 4.2864 | aux_loss: 0.0000
training loss: 4.2816 | aux_loss: 0.0000
training loss: 4.2844 | aux_loss: 0.0000
training loss: 4.2794 | aux_loss: 0.0000
training loss: 4.2843 | aux_loss: 0.0000
training loss: 4.2825 | aux_loss: 0.0000
training loss: 4.2843 | aux_loss: 0.0000
training loss: 4.2793 | aux_loss: 0.0000
training loss: 4.2811 | aux_loss: 0.0000
training loss: 4.2791 | aux_loss: 0.0000
training loss: 4.2810 | aux_loss: 0.0000
training loss: 4.2792 | aux_loss: 0.0000
training loss: 4.2946 | aux_loss: 0.0000
training loss: 4.2891 | aux_loss: 0.0000
training loss: 4.2932 | aux_loss: 0.0000
training loss: 4

KeyboardInterrupt: ignored

In case we want to use previously trained weights, to continue the training in the point we left it, we should load them into the model.

This is very useful in Google Colaboratory, that usually kills the virtual machine that is executing the Jupyter notework after a certime amount of time. If this happens to you, you should have to look for the last weights file in your configured Drive account and use it to train the network.


In [ ]:
# In case we want to use previously trained weights
weights = ""
if(len(weights)>0): model.load_weights(weights)

**Configure model**

In [ ]:
filepath = "/content/drive/My Drive/ISPR_project/LSTM{epoch:02d}-{loss:.4f}.h5"

checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=0,
                             save_best_only=True,mode='min')

callbacks_list = [checkpoint]

**Train!**

In [ ]:
prova=2500
prediction = model.predict(network_input[prova], verbose=0)
index = np.argmax(prediction)
index2 = np.argmax(network_output[prova])
print(index)
print(index2)

result = int_to_note[index]
print('\r', 'Predicted ', i, " ",result, end='')

60
52
 Predicted  4992   F#3

In [ ]:
model.fit(network_input, network_output, epochs=epochs, batch_size=8, callbacks=callbacks_list)

Epoch 1/100
625/625 [==============================] - 5s 8ms/step - loss: 4.1225
Epoch 2/100
625/625 [==============================] - 5s 8ms/step - loss: 3.7071
Epoch 3/100
625/625 [==============================] - 5s 8ms/step - loss: 3.6514
Epoch 4/100
625/625 [==============================] - 5s 8ms/step - loss: 3.6245
Epoch 5/100
625/625 [==============================] - 5s 8ms/step - loss: 3.6052
Epoch 6/100
625/625 [==============================] - 5s 8ms/step - loss: 3.5961
Epoch 7/100
148/625 [======>.......................] - ETA: 3s - loss: 3.5663

KeyboardInterrupt: ignored

**Music generation**

In [ ]:
# In case we want to use other previously trained weights
weights = "path/to/weights"
if(len(weights)>0): model.load_weights(weights)

ValueError: ignored

In [ ]:
# Generate network input again
network_input = []
output = []
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length]
  sequence_out = notes[i + sequence_length]
  network_input.append([note_to_int[char] for char in sequence_in])
  output.append(note_to_int[sequence_out])
n_patterns = len(network_input)

The workflow now is:


1.   Pick a **seed sequence** randomly from your list of inputs (*pattern* variable)
2.   Pass it as input for your model to generate a new element (note or chord)
3.   Add the new element to your final song and to your *pattern* list
4.   Remove the first item from *pattern*
5.   Go to step 2


In [ ]:
""" Generate notes from the neural network based on a sequence of notes """
# pick a random sequence from the input as a starting point for the prediction
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for i,note_index in enumerate(range(10)):
  prediction_input = np.reshape(pattern, (1, len(pattern), 1))
  prediction_input = prediction_input / float(n_vocab)
  prediction = model.predict(prediction_input, verbose=0)
  index = np.argmax(prediction)
  result = int_to_note[index]
  print('\r', 'Predicted ', i, " ",result, end='')
  prediction_output.append(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

 Predicted  9   G4

The last step is creating a MIDI file from the predictions.

**music21** will help us again for this task. We should create a **Stream** and add to it the predicted notes and chords.

We are adding an offset of 0.5 between elements.

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'